# [Assessing the clinical utility of cancer genomic and proteomic data across tumor types](http://www.nature.com/nbt/journal/v32/n7/full/nbt.2940.html)


* cancer의 molecular data를 임상적으로 활용하는데 얼마나 유용할 것인가에 대한 insight를 주는 논문임.
* survival 예측에 관련된 data resource 및 code를 **open-source로 제공함**으로써 각각의 모델들을 inspect, rerun, or improve 할 수 있는 계기가 되고자 함. 각각의 모델에 대한 C-index score를 real-time으로 leaderboard에 display.

### 1. Used data
https://www.synapse.org/#!Synapse:syn1710282/wiki/27303

Used data | TCGA 953명 (4개 cancer type)
----------------------|------------------------------------------------------------
clinical data | 성별, 나이, tumor stage & grade
molecular data| (1)somatic copy-number-alteration(CNA), (2)DNA methylation, (3)mRNA, (4)miRNA, (5)protein expression

4 cancer type| name| -
---------------------------|------------|-------------------------
GBM| glioblastoma multiforme| Brain-교모세포종
KIRC| Kidney renal clear cell carcinaoma| 신장암
OV|Ovarian serous cystadenocarcinoma| 장액성 난소암
LUSC| Lung squanmous cell carcinoma| 폐암
- 이들 4가지 cancer type들은 임상 정보 및 충분한 시간 동안의 survival data를 포함하고 5가지 molecular data중에 적어도 4개 이상 가지는 cancer들임

![table1](figures/table1.png)

### 2. Purpose & Results
Purpose| Result
----------------|------------------
survival 예측|3개 cancer type에서 비록 미세하지만 예측력(predictive power)을 증가시킴.(2.2~ 23.9%, FDR <0.05)
임상적으로 의미있는 유전자들에서의 somatic 변이 패턴 분석|총 12개 cancer type의 3,277명 환자 중 2,928명(89.4%)에서 10,281개 somatic mutation(SNV/InDel) 발견. 이 정보들은 치료 타겟을 설정하는데 도움이 될 것임




## Results1 
Assessment of the prognostic power of diverse molecular data

- 각 4 cancer type & feature set에 대해서
- core sample set을 정하고 100번씩 random splitting
- training set(80%) 과 test set(20%) 으로 구분하여 model을 만들고
- **c-index**로 예측력을 평가함.

- 2가지 model
    - Cox, the multivariate Cox proportional hazards model with L1 penalized log partial likelihood(LASSO)
    - RSF(random survival forest)

C-index? 예측 값과 실제 값 사이의 correlation값으로써 최대값 1일 경우 perfact 예측, 0.5일 경우 random하게 찍는 것과 동일한 예측력. 
(prediction error = 1 - C)

![fig1](figures/fig1.jpg)
![test](figures/test.png)

## Results2
Biological insights from top-performing prognostic models

* Nonnegative Matrix Factorization (NMF) algorithms ? (비음수 행렬 분해)

차원을 축소하여 노이즈 제거, 계산 비용축소, 정보의 특징을 추출하고자 하는 알고리즘. (이와 유사한 알고리즘- PCA)
![tmp1](figures/tmp1.png)

여기서 W의 열 벡터들은 V의 특징을 추출한 벡터로 간주. 행렬을 특징x계수로 어림 잡아 분해하여 **정보의 특징을 추출하는데 쓰인다.**

Log rank test? 서로 다른 survival curve끼리 비교하는 test. 모든 시점에서의 death를 group간에 비교하여 odds ratio를 계산한 후에 Mantel-Haenszel 방법으로 합쳐서 p-value계산

![fig2](figures/fig2.jpg)

- independent cohort (OV miRNA - survival data)
![tmp3](figures/tmp3.png)

- LUSC protein expression
- pMEK1, pMAPK, pS6 까지 이어지는 protein들이 shorter survival 환자 group(clusters2/3)에서 expression이 높음 (참고로 MEK inhibitor 약물이 non-small cell lung cancer (ex. KRAS-mutated lung adenocarcinoma) 에서 효과가 있음이 알려져 있음)
- cluster3에서 mTOR, src pathway가 activation되어있음을 관찰함

## Result3
Patient survival prediction using cross-tumor models

다른 cancer type으로 만든 model로 다른 type의 cancer의 survival을 예측할 수 있을까?
- 대부분의 경우 C-index가 0.5 근처
- 단, OV SCNA data로 training한 model은 KIRC 환자의 survival을 어느정도 예측(C-index 0.67)
![tmp4](figures/tmp4.png)

![fig3](figures/fig3.jpg)

- OV model(total n=379) 은 오히려 **KIRC** (training set n=192) 으로 만든 model보다 **KIRC** 예측을 더 잘함.
- OV model이 sample 수가 많고 같은 수 만큼 random하게 뽑아서 만든 model은 예측력이 약함.
- 따라서 OV model이 KIRC 예측을 잘 하는 이유는 larger sample size때문임.

Fig3b **arm-level SCNA features using GISTIC2**
- Features NOT selected by OV all vs. Features selected by OV all 
- (이 두 그룹의 q value를 비교하면, OV all에서 선택 된 features들의 q value가 유의미하게 작음 (p = 0.0016, wilcoxon rank sum test))

## Result4
Factors affecting prediction of **dichotomized survival data**

-어떤 factor가 survival data 예측에 영향을 많이 줄 것인지

- dichotomized survival data vs. censored survival 
    - dichotomized? 특정 cutoff time을 기준으로 이분화 시킴.
        - The dichotomized survival data based on the following cutoff:
        - KIRC: 4 years or 365 * 4= 1460 days, sample count: 150
        - GBM: 1 year or 365 days, sample count: 155
        - LUSC: 2 years or 365 * 2= 730 days, sample count: 77
        - OV: 3 years or 365 * 3= 1095 days, sample count: 252
    
    - censored? 사건 발생 여부에 대한 불확실한 자료를 censored data라고 부른다.
        - 추적이 불가능한 경우, 중도에 탈락된 경우, 연구의 종결, 관계없는 원인으로 사망하는 경우 등..
        - 이 정보는 비록 불확실하지만 censored 되기 직전까지는 사건이 발생하지 않았다는 부분적인 정보를 지니고 있으므로 이러한 점을 최대로 이용하여 분석함.
![censoring](figures/fig_survival.png)

- censored survival data를 특정 cutoff time을 기준으로 이분화 시킨 후 molecular data, clinical 정보를 이용하여 10-fold cross-validation으로 5,000 개가 넘는 model의 performance를 threashold-independent AUC score를 바탕으로 평가함.
        
- 예측력은 cancer type에 많이 의존함.
- 대체적으로 clinical variable이 molecular data에 비해 더 좋은 performance를 보임.(LUSC protein expression 제외)      

algorithm| -
---------|---
DDA| diagnal discriminant analysis
KNN| K-nearest neighbor
DA | discriminant analysis
LR | logistic regression
NC | nearest centroid
PLS| partial least squares
FR | random forest
SVM| support vector machine




![fig4](figures/fig4.jpg)

![fig5](figures/fig5.jpg)